## Wetland Dynamics in the Zambezi Delta, Mozambique  

This notebook demonstrates a methodology workflow for assessing wetland dynamics within the Zambezi River Delta, Mozambique, using a combination of satellite data products from Digital Earth Africa (DE Africa). It uses Landsat 8 Surface Reflectance (SR) and Fractional Cover (FC) product, along with Water Observations from Space (WOfS) product to derive key indicators of wetland health and change over time.  

**Motivation:**  
The Zambezi Delta is one of Africa's largest wetland ecosystems, supporting diverse biodiversity and local livelihoods. Understanding the dynamics is needed for effective water resource management, conservation efforts, and in other applictaion areas such public health risk assessment.  

**Methodology Overview:**  
The notebook follows these key steps:  
1.  **Data Access:** Connects to the DE Africa STAC catalog to retrieve Landsat 8 SR, WOfS, and Fractional Cover datasets for the Zambezi Delta.
2.  **Landsat SR Preprocessing:** Applies cloud masking and computes the Tasseled Cap Wetness (TCW) index.
3.  **WOfS:** Utilizes DE Africa's pre-computed Water Observations from Space (WOfS) product.
4.  **Fractional Cover Analysis:** Masks Fractional Cover data using WOfS and TCW to focus on land cover dynamics in non-open water/non-wet areas.
5.  **Time-series Analysis:** Calculates the percentage area of open water, wetness, green vegetation, dry vegetation, and bare soil over time.
6.  **Visualization:** Generates a stacked area plot showing the temporal distribution of these land cover components and an animated GIF visualizing the spatial changes.


**Acknowledgement:**  
This notebook adapts and integrates methods from various Digital Earth Africa example/guide notebooks, specifically on STAC access, Landsat preprocessing, and Fractional Cover analysis.

### Prerequisites

To run this notebook, the following Python libraries should be installed in your environment:  
`pip install deafricatools pystac-client odc-stac geopandas xarray rioxarray scikit-image matplotlib seaborn pandas`

Additionally, the `get_product_config.py` file should be included in the same repository. This custom utility function is needed for retrieving the specific STAC configuration details needed to correctly load different Digital Earth Africa products.

### 1. Environment Setup

This section sets up the environment by loading the relevant Python libraries and configuring the connection to the Digital Earth Africa STAC catalog and AWS S3 for data access.

#### 1.1 Load Libraries

In [1]:
import numpy as np                              # For numerical operations 
import geopandas as gpd                         # For working with geospatial vector data 
import matplotlib as plt                        # For creating static, animated, and interactive plots
import rioxarray                                # For raster I/O with xarray
import xarray as xr                             # For working with multi-dimensional labeled datasets 
import datetime                                 # For manipulating dates and times
import pprint                                   # For pretty-printing data structures
import seaborn as sns                           # For statistical data visualization
import pandas as pd                             # For dataframes manipulation 
import matplotlib.animation as animation        # For creating animations in matplotlib
import matplotlib.patheffects as PathEffects    # For adding effects to plot text
import odc.algo                                 # For analysis functions from the Open Data Cube (ODC) algorithm library

from mpl_toolkits.axes_grid1.inset_locator import inset_axes            # For adding inset axes to plots
from shapely.geometry import box, Polygon                               # For creating and manipulating geometric shapes
from get_product_config import get_product_config                       # Custom function to retrieve product-specific configuration (user-defined)
from odc.stac import stac_load, configure_rio                           # For loading and configuring STAC items for analysis via ODC
from pystac_client import Client                                        # For accessing STAC metadata using the PySTAC Client
from deafrica_tools.datahandling import mostcommon_crs, wofs_fuser      # For loading and handling DE Africa data (ARD, CRS, WOfS)
from deafrica_tools.plotting import rgb, display_map, plot_wofs         # For visualizing satellite imagery and WOfS outputs
from datacube.utils import masking, geometry                            # For masking data and handling spatial geometry in the DataCube
from datacube.utils.geometry import CRS                                 # For defining coordinate reference systems (CRS)
from deafrica_tools.bandindices import calculate_indices                # For computing band indices (e.g., NDVI, NDBI)
from deafrica_tools.classification import HiddenPrints                  # For suppressing print output in code blocks
from deafrica_tools.spatial import xr_rasterize                         # For rasterizing vector geometries into xarray datasets


#### 1.2 Connect to Digital Earth Africa STAC Catalog

This configures the connection to the Digital Earth Africa STAC endpoint. The `configure_rio` function optimizes data access from AWS S3, ensuring efficient loading of satellite imagery.

In [ ]:
# configure aws 
configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
    AWS_s3_ENDPOINT="s3.af-south-1.amazonaws.com",
)

#open STAC
catalog = Client.open("https://explorer.digitalearth.africa/stac")



### 2. Utility Functions for Data Access

The following helper functions are defined , for effective analysis. These functions are the common steps for retrieving product configurations, querying the catalog, and loading the data into xarray.Dataset objects.

#### 2.1 Get Product Configuration 

This function retrieves the specific configuration details for a given DE Africa product (e.g., `ls8_sr`, `wofs_ls`, `fc_ls`).

In [ ]:
# set collection configuration
def get_config(product_name, profile="deafrica"):
    config = get_product_config(product_name, profile)
    pprint.pprint(config)
    return config

#### 2.2 Query STAC Catalog 
This function builds and executes a query for the connected STAC catalog to find available datasets (items) within a specified bounding box and time range for a particular product.

In [ ]:
# Build query with set parameters to search the STAC catalogue
def search_stac_catalog(catalog, product_name, bbox, start_date, end_date):
    query = catalog.search(
        bbox=bbox,
        collections=[product_name],
        datetime=f"{start_date}/{end_date}"
    )
    items = list(query.items())
    print(f"Found: {len(items)} datasets")
    return items

##### 2.3 Load STAC Items
This function loads the found STAC items into an xarray.Dataset, with specific parameters.

In [ ]:
# load the required dataset

def load_stac_items(
    items,
    bbox,
    config,
    crs="EPSG:32636",
    resolution=30,
    bands=None,
    chunks={},
    groupby=None,
    fuse_func=None,
    collection_category=None
):
    return stac_load(
        items=items,
        bands=bands,
        crs=crs,
        resolution=resolution,
        chunks=chunks,
        groupby=groupby,
        fuse_func=fuse_func,
        collection_category=collection_category,
        stac_cfg=config,
        bbox=bbox,
    )


### 3. Define Analysis Parameters and Area of Interest

This section defines the temporal and spatial parameters for the analysis. The *bbox* (bounding box) specifies the geographic area of interest within the Zambezi Delta.

The coordinates *lat*, *lon* (latitude, longitude) define the center of the study area, and *buffer* determines the size of the bounding box around this center. This can be adjusted to focus on different parts of the delta.

In [ ]:
# Time range
start_date = "2017"
end_date = "2024"

# Bounding box around Mopeia area
lat, lon = -17.97757, 35.7130
buffer = 0.177
bbox = [lon - buffer, lat - buffer, lon + buffer, lat + buffer]

# Projection and resolution
crs = "EPSG:32636"
resolution = 30


### 4. Landsat 8 Surface Reflectance Processing

This section focuses on loading the Landsat 8 SR data, applying cloud masking preprocessing step and computing the Tasseled Cap Wetness (TCW) index.

#### 4.1 Load Landsat 8 SR Data

We load the Landsat 8 Collection 2 Surface Reflectance product (`ls8_sr`) from the DE Africa STAC catalog. This product includes various spectral bands (Red, Green, Blue, NIR, SWIR1, SWIR2) and a `pixel_quality` band that is necessary for cloud masking.


In [ ]:
# Load Landsat 8 SR
config_ls = get_config("ls8_sr")

items_ls = search_stac_catalog(catalog, 
                               "ls8_sr", 
                               bbox, 
                               start_date, 
                               end_date)

ds_ls = stac_load(
    items_ls,
    bands=("red", "green", "blue", "nir", "swir_1", "swir_2", "pixel_quality"),
    crs="EPSG:32636",
    resolution=30,
    chunks={},
    groupby="solar_day",
    stac_cfg=config_ls,
    bbox=bbox,
)

# View the xarray.Dataset.
ds_ls

In [ ]:
# view the dataset
rgb(ds_ls.isel(time=[0,1,2]),bands=['red', 'green', 'blue'], 
    col='time')

### 4.2 Cloud Masking

Cloud masking is an important preprocessing step to ensure that the analysis is not affected by atmospheric interference. The `pixel_quality` band which contains bit flags indicating various pixel classifications (e.g., cloud, cloud shadow, cirrus, snow, water, clear) is utilized for the masking

The flags_definition attribute to the pixel_quality band using the [product definition on DE Africa Explorer](https://explorer.digitalearth.africa/products/ls8_sr) is applied manually.

In [ ]:
# manually assign attributes to the pixel_quality band
ds_ls["pixel_quality"].attrs = {"units": "bit_index",
                           "nodata": 1,
                           "flags_definition": {
                               "nodata": {"bits": 0,"values": {0: False, 1: True}},
                               "dilated_cloud": {"bits": 1,"values": {0: "not_dilated", 1: "dilated"}},
                               "cirrus": {"bits": 2,"values": {0: "not_high_confidence", 1: "high_confidence"}},
                               "cloud": {"bits": 3,"values": {0: "not_high_confidence", 1: "high_confidence"}},
                               "cloud_shadow": {"bits": 4,"values": {0: "not_high_confidence", 1: "high_confidence"}},
                               "snow": {"bits": 5,"values": {0: "not_high_confidence", 1: "high_confidence"}},
                               "clear": {"bits": 6,"values": {0: False, 1: True}},
                               "water": {"bits": 7,"values": {0: "land_or_cloud", 1: "water"}},
                               "cloud_confidence": {"bits": [8, 9],"values": {0: "none", 1: "low", 2: "medium", 3: "high"}},
                               "cloud_shadow_confidence": {"bits": [10, 11],"values": {0: "none", 1: "low", 2: "reserved", 3: "high"}},
                               "snow_ice_confidence": {"bits": [12, 13],"values": {0: "none", 1: "low", 2: "reserved", 3: "high"}},
                               "cirrus_confidence": {"bits": [14, 15],"values": {0: "none", 1: "low", 2: "reserved", 3: "high"}}
                               }
                        }


In [ ]:
# apply cloud masking on the landsat data
from datacube.utils.masking import make_mask

def mask_landsat_pq(ds):
    """
    Masks Landsat 8 pixels with any cloud, cloud shadow, cirrus, snow, dilated cloud, or nodata bits set.

    Parameter
        ds(xarray.Dataset) : Landsat dataset with 'pixel_quality' band.

    Returns
        xarray.Dataset: Dataset with bad pixels masked out (set to NaN).
    """
    cloud_bit = 1 << 3
    cloud_shadow_bit = 1 << 4
    cirrus_bit = 1 << 2
    snow_bit = 1 << 5
    dilated_cloud_bit = 1 << 1
    nodata_bit = 1 << 0

    bad_bits = (cloud_bit | cloud_shadow_bit | cirrus_bit | snow_bit | dilated_cloud_bit | nodata_bit)

    mask = (ds.pixel_quality.data & bad_bits) == 0

    return ds.where(mask)

# Apply the masking
ds_masked = mask_landsat_pq(ds_ls)


In [ ]:
# visualize
rgb(ds_masked.isel(time=[0,1,2]),bands=['red', 'green', 'blue'], col='time',col_wrap=6,
    size=6)

In [ ]:
# Checking if the cloud data pixels are properly masked and represented as NaN values
red_orig = ds_ls['red'].isel(time=0)
red_masked = ds_masked['red'].isel(time=0)

print("Original red band NaNs:", np.isnan(red_orig).sum().compute())
print("Masked red band NaNs:", np.isnan(red_masked).sum().compute())


### 4.3 Compute Tasseled Cap Wetness (TCW) Index

The Tasseled Cap Wetness (TCW) index is an index from TCT (Tasseled Cap Transformation) that uses Landsat bands to detect soil and vegetation moisture.

**Steps involved:**
1.  **Create AOI Mask:** A spatial mask for the defined AOI is created to ensure computations are confined to the study area.
2.  **Normalize Landsat Data:** Landsat surface reflectance values, scaled by 10000, are normalized to a 0-1 range before index computation.
3.  **Compute TCW:** The `calculate_indices` function from `deafrica_tools` is used to compute the raw TCW values.
4.  **Otsu Thresholding:** Otsu's method is applied to the raw TCW values to automatically determine an optimal threshold that separates "wet" from "dry" pixels. 
5.  **Monthly Resampling:** The TCW data is resampled to a monthly maximum. 
6.  **Binary Mask:** The resampled TCW is converted into a binary mask where `True` indicates "wet" pixels (above the Otsu threshold) and `False` indicates "dry" pixels.
 

In [ ]:
# Rasterize AOI and mask ls8_sr
def create_aoi_mask(ds, verbose=True):
    """
    Create a rasterized AOI mask using the global bbox, lat, lon, buffer, and crs.

    Parameters:
        ds (xarray.Dataset): A template dataset to match dimensions.
        verbose (bool): Print progress messages.

    Returns:
        xarray.DataArray: AOI mask aligned with the input dataset.
    """
    global lat, lon, buffer, crs

    if verbose:
        print("Creating AOI polygon and raster mask...")

    bbox_coords = [
        (lon - buffer, lat - buffer),
        (lon + buffer, lat - buffer),
        (lon + buffer, lat + buffer),
        (lon - buffer, lat + buffer),
        (lon - buffer, lat - buffer)
    ]
    gdf = gpd.GeoDataFrame({"aoi": ["target"]},
                           geometry=[Polygon(bbox_coords)],
                           crs="EPSG:4326").to_crs(crs)

    template = ds.isel(time=0)
    mask = xr_rasterize(gdf, template)

    if verbose:
        print("AOI mask created.")
    return mask

In [ ]:
# compute TCW 
from skimage.filters import threshold_otsu

def compute_tcw(ds, mask, verbose=True):
    """
    Compute the TCW index and apply an Otsu threshold (to determine tcw threshold from the raw TCW values).

    Parameters:
        ds (xarray.Dataset): Normalized dataset
        mask (xarray.DataArray): AOI mask to apply to the result.

    Returns:
        xarray.DataArray: Binary TCW mask (wet = True).
    """
    if verbose:
        print("Computing TCW index...")

    with HiddenPrints():  
        tcw_raw = calculate_indices(
            ds, 
            index=["TCW"], 
            normalise=False, 
            satellite_mission="ls", 
            drop=True
        )

    if verbose:
        print("Calculating Otsu threshold...")

    # define threshold
    flat = tcw_raw.TCW.values.flatten()
    flat = flat[~np.isnan(flat)]
    otsu_thresh = threshold_otsu(flat) 

    if verbose:
        print(f"Otsu threshold: {otsu_thresh:.4f}")
        print("Resampling TCW to monthly max...")
 
    # resample monthly
    tcw_resampled = tcw_raw.resample(time="1M").max() 
    tcw = tcw_resampled.TCW >= otsu_thresh
    tcw = tcw.where(mask, 0).persist()

    if verbose:
        print("TCW binary mask created.")
    return tcw

In [ ]:
# Create AOI mask aligned with ds_ls
mask = create_aoi_mask(ds_ls)

# Apply AOI mask to the original ds_ls
ds_ls_masked = ds_ls.where(mask)

# Normalize the masked data
ds_ls_normalized = ds_ls_masked / 10000

# Compute TCW index on normalized, masked data
tcw = compute_tcw(ds_ls_normalized, mask)

In [ ]:
# visualize tcw
# 1 = wet and 0 = dry
tcw.isel(time=[0,1,2,3,4,5]).plot.imshow(col='time',col_wrap=3,cmap='Blues', size=4); 


### 5. Water Observations from Space (WOfS) Processing

Digital Earth Africa's Water Observations from Space (WOfS) is a value-added product from historical Landsat that shows water presence/persistence.

#### 5.1 Load WOfS Data

Load the WOfS Landsat product (`wofs_ls`) for the study area. 


In [ ]:
# Load WOfS
config_wofs = get_config("wofs_ls")

items_wofs = search_stac_catalog(catalog, 
                               "wofs_ls", 
                               bbox, 
                               start_date, 
                               end_date)

ds_wofs = stac_load(
    items_wofs,
    crs="EPSG:32636",
    resolution=30,
    chunks={},
    fuse_func= wofs_fuser,
    collection_category="T1",
    stac_cfg=config_wofs,
    bbox=bbox,
)

# View the xarray.Dataset.
ds_wofs

In [ ]:
# visualize wofs
ds_wofs.water.isel(time=[0,1,2,3,4,5]).plot.imshow(col='time',col_wrap=3,cmap='Blues', size=4); 

#### 5.2 Process WOfS Data for Analysis

Similar to Landsat, WOfS data also contains specific pixel quality flags. We manually attach these flags_definition attributes to the water band, aligning with the [WOfS product definition on DE Africa Explorer](https://explorer.digitalearth.africa/products/wofs_ls).

The `process_wofs_mask` function then:
1.  Creates a boolean mask for wet pixels from the WOfS water band.
2.  Resamples the WOfS wet mask to a monthly maximum, aligning its time dimension with the **TCW** data for consistent temporal analysis.
3.  Applies the previously created AOI mask.

In [ ]:
# Attach the WOfS flag definitions manually
ds_wofs.water.attrs["flags_definition"] = {
    "wet": {"bits": [7, 6, 5, 4, 3, 2, 1, 0],"values": {"128": True},"description": "Clear and wet"},
    "dry": {"bits": [7, 6, 5, 4, 3, 2, 1, 0],"values": {"0": True},"description": "No water detected"},
    "cloud": {"bits": 6,"values": {"1": True},"description": "Cloudy"},
    "nodata": {"bits": 0,"values": {"1": True},"description": "No data"},
    "water_observed": {"bits": 7,"values": {"1": True},"description": "Classified as water by the decision tree"}}

In [ ]:
# apply tcw mask and resample monthly
def process_wofs_mask(ds_wofs, tcw, mask, verbose=False):
    """
    Converts WOfS water observations to a monthly boolean wet mask,
    aligned to a reference dataset's time and spatial mask.

    Parameters
    ----------
    ds_wofs : xarray.Dataset
        Dataset containing the 'water' band from WOfS.
    tcw : xarray.DataArray or Dataset
        Reference dataset used to match the time dimension.
    mask : xarray.DataArray
        Spatial mask with True where data should be retained.
    verbose : bool
        If True, prints progress info.

    Returns
    -------
    xarray.DataArray
        Boolean wet mask, monthly resampled, aligned to tcw and spatially masked.
    """

    # Create wet/dry boolean mask from WOfS water band
    wofs_mask = masking.make_mask(ds_wofs.water, wet=True)

    # Resample to monthly frequency
    if verbose:
        print("Resampling WOfS to 1M")
    wofs_wet = wofs_mask.resample(time="1M").max()

    # Align time with TCW
    wofs_wet = wofs_wet.sel(time=tcw.time)

    # Apply spatial mask
    wofs_wet = wofs_wet.where(mask)

    return wofs_wet

# apply 
wofs_wet = process_wofs_mask(ds_wofs, tcw=tcw, mask=mask, verbose=True)


In [ ]:
# visualize wofs mask
wofs_wet.isel(time=[0,1,2,3,4,5]).plot.imshow(col='time',col_wrap=3,cmap='Blues', size=4) 

### 6. Fractional Cover (FC) Processing

Digital Earth Africa's Fractional Cover (FC) product provides estimates of the proportion of ground covered by photosynthetic vegetation (PV), non-photosynthetic vegetation (NPV), and bare soil (BS) for every pixel. 

#### 6.1 Load Fractional Cover Data

We load the Landsat Fractional Cover product (`fc_ls`) for the study area. 

In [ ]:
# Load FC
config_fc = get_config("fc_ls")

items_fc = search_stac_catalog(catalog, 
                               "fc_ls", 
                               bbox, 
                               start_date, 
                               end_date)

ds_fc_ls = stac_load(
    items_fc,
    crs="EPSG:32636",
    resolution=30,
    bands = ("pv", "npv", "bs"),
    chunks={},
    collection_category="T1",
    stac_cfg=config_fc,
    bbox=bbox,
)

# View the xarray.Dataset.
ds_fc_ls

In [ ]:
# visualize fractional cover
rgb(ds_fc_ls.isel(time=[0,1,2,3,4,5]),  # Select first 5 time steps
    col='time',
    bands=['bs', 'pv', 'npv'],
    col_wrap=6,
    size=6)

#### 6.2 Mask Fractional Cover Data

To accurately assess vegetation dynamics in *non-inundated* areas of the wetland, the masks derived from WOfS and TCW are applied. This ensures that areas identified as open water or general wetness are excluded from the fractional cover analysis, allowing us to focus on the changes in vegetation type on relatively drier or temporarily wet surfaces.

The `preprocess_fc` function performs the following:
1.  **Mask `clear_and_dry` WOfS pixels:** It retains only pixels that WOfS identifies as clear and dry, filtering out open water.
2.  **Monthly Resampling:** Resamples the FC data to a monthly maximum.
3.  **Time Alignment:** Aligns the FC data's time dimension with that of the TCW data.
4.  **Apply AOI and Wetness Mask:** Applies the overall AOI mask and *excludes* pixels identified as wet by the TCW index.

In [ ]:
def preprocess_fc(ds_wofs, ds_fc_ls, tcw, mask, verbose=False):
    if verbose:
        print("Masking clear and dry WOfS pixels")
    clear_and_dry = masking.make_mask(ds_wofs, dry=True).water
    ds_fc_clear = ds_fc_ls.where(clear_and_dry)
    
    if verbose:
        print("Resampling FC to monthly max")
    ds_fc_resamp = ds_fc_clear.resample(time="1M").max()
    
    if verbose:
        print("Matching FC to TCW time")
    ds_fc_matched = ds_fc_resamp.sel(time=tcw.time)
    
    if verbose:
        print("Applying AOI and wetness mask")
    ds_fc_masked = ds_fc_matched.where(mask)
    ds_fc_masked = ds_fc_masked.where(tcw == False)  # noqa: E712

    return ds_fc_masked


fc_ds_noTCW = preprocess_fc(ds_wofs,ds_fc_ls,tcw,mask,verbose=True)

##### Visualize Masked Fractional Cover Data

This visualization shows the Fractional Cover data after masking out open water and very wet areas. The color scheme represents:
* **Red tones:** Bare Soil (BS)
* **Green tones:** Photosynthetic Vegetation (PV)
* **Blue tones:** Non-Photosynthetic Vegetation (NPV)

In [ ]:
rgb(fc_ds_noTCW.isel(time=[0,1,2,3,4,5]),  # Select first 5 time steps
    col='time',
    bands=['bs', 'pv', 'npv'],
    col_wrap=3,
    size=4)

### 7. Derive Wetland Classes and Calculate Area Percentages

This section combines the processed WOfS, TCW, and Fractional Cover data to derive the dominant land cover classes for each pixel over time, and then calculates the percentage area covered by each class within the defined Zambezi Delta AOI.

**Derived Classes:**
* **Open Water:** Directly from WOfS wet pixels.
* **Wetland (non-open water):** Areas identified as wet by TCW but not classified as open water by WOfS. These often represent vegetated wetlands or saturated soils.
* **Green Vegetation (PV):** Areas dominated by photosynthetic vegetation, derived from Fractional Cover.
* **Dry Vegetation (NPV):** Areas dominated by non-photosynthetic vegetation, derived from Fractional Cover.
* **Bare Soil (BS):** Areas dominated by bare ground, derived from Fractional Cover.

#### 7.1 Classify Dominant Fractional Cover

The `classify_fc` function determines the dominant Fractional Cover component (PV, NPV, or BS) for each pixel based on which band has the highest value.



In [ ]:
# dominant fc cover class
def classify_fc(fc_data_array):
    """
    Classifies each pixel in the Fractional Cover dataset into its dominant component.

    Parameters:
        fc_data_array (xarray.Dataset): Fractional Cover dataset with 'pv', 'npv', 'bs' bands.

    Returns:
        xarray.Dataset: A new dataset with boolean masks for 'pv', 'npv', 'bs' for each pixel.
    """
    fc_ds_noTCW = fc_data_array.to_array(dim="variable", name="fc_ds_noTCW").astype("int8")
    argmax = fc_ds_noTCW.argmax(dim="variable")
    fc_mask = np.isfinite(fc_ds_noTCW).all(dim="variable")
    argmax = argmax.where(fc_mask)
    
    return xr.Dataset(
        {
            "bs": (argmax == 2).where(fc_mask),
            "pv": (argmax == 0).where(fc_mask),
            "npv": (argmax == 1).where(fc_mask),
        }
    )
# Restack the Fractional cover dataset all together
    # CAUTION:ARGMAX DEPENDS ON ORDER OF VARIABALES IN
    # DATASET. NEED TO ADJUST BELOW DEPENDING ON ORDER OF FC VARIABLES
    
FC_dominant = classify_fc(fc_ds_noTCW)

In [ ]:
# count pixels
def compute_pixel_counts(mask, tcw, FC_dominant, wofs_wet, verbose=True):
    """
    Computes the total number of pixels in the AOI and the pixel counts
    for the land cover classes (wet, green veg, dry veg, bare soil, open water).

    Parameters:
        mask (xarray.DataArray): The AOI mask.
        tcw (xarray.DataArray): Binary TCW wet mask.
        FC_dominant (xarray.Dataset): Dataset with boolean masks for dominant FC classes.
        wofs_wet (xarray.DataArray): Binary WOfS wet mask.
        verbose (bool): If True, prints progress messages.

    Returns:
        tuple: (total_pixels_in_AOI, tcw_pixel_count, FC_class_counts, wofs_pixel_count)
    """
    
    pixels = mask.sum(dim=["x", "y"])
    
    if verbose:
        print("Computing wetness")
    tcw_pixel_count = tcw.sum(dim=["x", "y"]).compute()

    if verbose:
        print("Computing green veg, dry veg, and bare soil")
    FC_count = FC_dominant.sum(dim=["x", "y"]).compute()

    if verbose:
        print("Computing open water")
    wofs_pixels = wofs_wet.sum(dim=["x", "y"]).compute()
    
    return pixels, tcw_pixel_count, FC_count, wofs_pixels


pixels, tcw_pixel_count, FC_count, wofs_pixels = compute_pixel_counts(
    mask, tcw, FC_dominant, wofs_wet, verbose=True)

In [ ]:
# get percentage cover and nodata
def calculate_percentages(pixels, tcw_pixel_count, FC_count, wofs_pixels):
    """
    Calculates the percentage cover for Open Water, Wetness (excluding open water),
    Green Vegetation, Dry Vegetation, and Bare Soil within the AOI.

    Parameters:
        pixels (xarray.DataArray): Total valid pixels in the AOI.
        tcw_pixel_count (xarray.DataArray): Pixel count for TCW wet areas.
        FC_count (xarray.Dataset): Pixel counts for dominant FC classes.
        wofs_pixels (xarray.DataArray): Pixel count for WOfS wet areas.

    Returns:
        tuple: (wofs_area_percent, tcw_less_wofs_percent, PV_percent, NPV_percent, BS_percent)
    """
    BS_percent = (FC_count.bs / pixels) * 100
    PV_percent = (FC_count.pv / pixels) * 100
    NPV_percent = (FC_count.npv / pixels) * 100
    wofs_area_percent = (wofs_pixels / pixels) * 100
    tcw_area_percent = (tcw_pixel_count / pixels) * 100
    tcw_less_wofs = tcw_area_percent - wofs_area_percent

    NoData_count = (
        (100 - wofs_area_percent - tcw_less_wofs - PV_percent - NPV_percent - BS_percent) / 100
    ) * pixels
    
    valid_pixels = (pixels - NoData_count).where((pixels - NoData_count) > 0)

    # Recalculate with valid pixels
    BS_percent = (FC_count.bs / valid_pixels) * 100
    PV_percent = (FC_count.pv / valid_pixels) * 100
    NPV_percent = (FC_count.npv / valid_pixels) * 100
    wofs_area_percent = (wofs_pixels / valid_pixels) * 100
    tcw_area_percent = (tcw_pixel_count / valid_pixels) * 100
    tcw_less_wofs = (tcw_area_percent - wofs_area_percent).where(lambda x: x >= 0, 0)
    
    return wofs_area_percent, tcw_less_wofs, PV_percent, NPV_percent, BS_percent

wofs_area_percent, tcw_less_wofs, PV_percent, NPV_percent, BS_percent = calculate_percentages(
    pixels, tcw_pixel_count, FC_count, wofs_pixels)

In [ ]:
# create and export datafarame
def create_export_dataframe(wofs_percent, tcw_percent, pv_percent, npv_percent, bs_percent, filename=None, verbose=False):
    """
    Creates a Pandas DataFrame summarizing percentage cover for different wetland classes
    and optionally exports it to a CSV file.

    Parameters:
        wofs_percent (xarray.DataArray): Percentage of open water.
        tcw_percent (xarray.DataArray): Percentage of wetness (excluding open water).
        pv_percent (xarray.DataArray): Percentage of green vegetation.
        npv_percent (xarray.DataArray): Percentage of dry vegetation.
        bs_percent (xarray.DataArray): Percentage of bare soil.
        filename (str, optional): Name of the CSV file to export. If None, no export.
        verbose (bool): If True, prints export messages.

    Returns:
        pandas.DataFrame: A DataFrame with time as index and percentage cover as columns.
    """
    df = pd.DataFrame(
        data=wofs_percent.data,
        index=wofs_percent.time.values,
        columns=["wofs_area_percent"],
    )
    df["wet_percent"] = tcw_percent.data
    df["green_veg_percent"] = pv_percent.data
    df["dry_veg_percent"] = npv_percent.data
    df["bare_soil_percent"] = bs_percent.data
    df = df.round(2)
    
    if filename:
        if verbose:
            print(f"Exporting CSV: {filename}")
        df.to_csv(filename, index_label="Datetime")

    return df

# create the data frame
df = create_export_dataframe(
    wofs_percent=wofs_area_percent,
    tcw_percent=tcw_less_wofs,
    pv_percent=PV_percent,
    npv_percent=NPV_percent,
    bs_percent=BS_percent,
    filename="wetland_summary.csv",
    verbose=True
)

In [ ]:
df.head(6)

### 8. Visualize Wetland Dynamics

This section presents the results of the analysis through two types of visualizations:
1.  A **stacked area plot** showing the temporal distribution of different land cover classes (open water, wetness, vegetation, bare soil) as percentages over time.
2.  An **animated GIF** illustrating the spatial changes in land cover (including inundation) over the Zambezi Delta throughout the analysis period.

#### 8.1 Time-Series Stacked Area Plot

This plot provides a high-level overview of how the proportion of open water, wet areas (non-open water), green vegetation, dry vegetation, and bare soil has changed in the Zambezi Delta from 2017 to 2024. This highlights seasonal patterns and inter-annual variations.

In [ ]:
import matplotlib.pyplot as plt

export_plot = True
out_filename = 'data/output/zambezi_wet3.png'

# generate plot
#set up color palette
pal = [sns.xkcd_rgb["cobalt blue"],
       sns.xkcd_rgb["neon blue"],
       sns.xkcd_rgb["grass"],
       sns.xkcd_rgb["beige"],
       sns.xkcd_rgb["brown"]]

#make a stacked area plot
plt.clf()
fig = plt.figure(figsize=(15,6))
plt.stackplot(df.index,
              df.wofs_area_percent,
              df.wet_percent,
              df.green_veg_percent,
              df.dry_veg_percent,
              df.bare_soil_percent,
              labels=['open water', 'wet', 'green veg', 'dry veg', 'bare soil'],
              colors=pal, alpha=0.6)

plt.axis(xmin=df.index[0], xmax=df.index[-1], ymin=0, ymax=100)
plt.tick_params(labelsize=14)
plt.legend(loc='lower left', framealpha=0.6, fontsize=14)
plt.title('Fractional Cover, Wetness and Water', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Area (%)', fontsize=14)
plt.tight_layout()

if export_plot:
    plt.savefig(out_filename)


#### 8.2 Animated Spatial Map

This animation provides a dynamic visual representation of how wetland inundation and land cover classes change spatially over time within the Zambezi Delta. It combines the Fractional Cover data with the WOfS wet mask to show a composite view.

**Color Representation:**
* **Blue:** Open Water (derived from WOfS)
* **Green tones:** Photosynthetic Vegetation (PV)
* **Brown tones:** Non-Photosynthetic Vegetation (NPV)
* **Redd tones:** Bare Soil (BS)

*(Note: Generating large GIFs can be memory-intensive and may take some time depending on the size of the AOI and time range.)*

In [ ]:
# same function but for just FC masked aoi 
def preprocess_fc(ds_wofs, ds_fc_ls, tcw, mask, verbose=False):
    if verbose:
        print("Masking clear and dry WOfS pixels")
    clear_and_dry = masking.make_mask(ds_wofs, dry=True).water
    ds_fc_clear = ds_fc_ls.where(clear_and_dry)
    
    if verbose:
        print("Resampling FC to monthly max")
    ds_fc_resamp = ds_fc_clear.resample(time="1M").max()
    
    if verbose:
        print("Matching FC to TCW time")
    ds_fc_matched = ds_fc_resamp.sel(time=tcw.time)
    
    if verbose:
        print("Applying AOI and wetness mask")
    ds_fc_masked = ds_fc_matched.where(mask)
    #ds_fc_masked = ds_fc_masked.where(tcw == False)  # noqa: E712

    return ds_fc_masked

fc_masked = preprocess_fc(ds_wofs,ds_fc_ls,tcw,mask,verbose=True)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Raise embed limit for view in notebook
mpl.rcParams['animation.embed_limit'] = 60  # MB

frames = range(0, len(fc_masked.time))

# Normalize FC layers (assumed 0–100%)
pv = fc_masked['pv'] / 100
npv = fc_masked['npv'] / 100
bs = fc_masked['bs'] / 100

# Use water mask (1 = water, 0 = no water, np.nan = no data)
water_mask = wofs_wet.fillna(0)  # Fill NaNs with 0 to avoid issues

# Get coordinate extents (for lat/lon display)
x_coords = fc_masked['x'].values
y_coords = fc_masked['y'].values
extent = [x_coords.min(), x_coords.max(), y_coords.min(), y_coords.max()]

# Set up figure
fig, ax = plt.subplots(figsize=(6, 6))
img = ax.imshow(np.zeros((pv.sizes['y'], pv.sizes['x'], 3)), extent=extent, origin='upper', vmin=0, vmax=1)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.tick_params(labelsize=8)  

def update(frame):
    r = 0.6 * npv[frame].values + 0.5 * bs[frame].values
    g = 0.8 * pv[frame].values + 0.3 * npv[frame].values
    b = 0.3 * bs[frame].values

    rgb = np.stack([r, g, b], axis=-1)

    # Overlay water: enhance blue channel
    water = water_mask[frame].values
    rgb[..., 2] = np.where(water == 1, 1.0, rgb[..., 2])   # blue
    rgb[..., 0] = np.where(water == 1, 0.0, rgb[..., 0])   # reduce red
    rgb[..., 1] = np.where(water == 1, 0.4, rgb[..., 1])   # slight green

    rgb = np.clip(rgb, 0, 1)

    img.set_data(rgb)
    ax.set_title(str(fc_ds_noTCW['time'][frame].values)[:10])
    return [img]

ani = FuncAnimation(fig, update, frames=frames, interval=300)
plt.close()

# Display in notebook
HTML(ani.to_jshtml())
